In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import cvxpy as cp
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt

from scipy import stats 
from scipy import interpolate
from sprd import grids, opt, bestof

In [3]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision = 4)
%matplotlib widget

In [4]:
# set up the grid for X and Y, and sgrid for the index
npts = 101
fwd_for_grid = 0.0
std_for_grid = 1.0
n_std = 3
xgrid = grids.gen_grid(npts, fwd = fwd_for_grid, stdev=std_for_grid, nstd = n_std)
sx,sy = np.meshgrid(xgrid,xgrid,indexing = 'ij')
sgrid = grids.gen_sgrid(xgrid)


### Calc or Load the data and optionally save it

In [5]:
filename = 'data/data_16_1_best_of_calls_arbitrage.xls'
load_data = False
save_data = False
test_plots = False

x0 = 0
y0 = 0

if load_data:
    xgrid, xdens, ydens, bdens,callsx, callsy, callsb = bestof.load_boc_calls_convert_to_densities(filename)
    npts = len(xgrid)
else:
    xvol = 1.0
    yvol = 0.75
    xycor = 0.0
    blend = 3
    xdens,ydens,bdens, callsx, callsy, callsb = bestof.gen_modified_gauss_boc_densities(
    xgrid,  x0,y0,xvol, yvol, xycor, blend = blend, from_cdf=True, test_plots = test_plots)

if save_data:
    bestof.save_boc_calls(xgrid, callsx, callsy, callsb, filename)


### Check if the problem is feasible
(unfeasible is good)

In [ ]:


# tolerance; equality makes this hard
eps = 1e-6

# 2d density
x = cp.Variable((npts,npts))
constraints = opt.get_boc_constraints(x, xdens, ydens, bdens, eps = eps)

# obj = cp.Minimize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0))
obj = cp.Maximize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0))
# obj = cp.Maximize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0) - smooth_obj)
prob = cp.Problem(obj, constraints)
# res = prob.solve()
res = prob.solve( solver = 'SCIPY', scipy_options = {'method':'highs'})
pij = x.value
print(prob.status)
print(prob.value)

In [7]:
check_plots = False
if check_plots:
    pij = x.value
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.plot_surface(sx, sy, pij, alpha=1.0, cmap = 'Reds')

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Joint density')
    plt.legend(loc = 'best')

### Dual

In [ ]:
f = cp.Variable((npts,))
g = cp.Variable(npts)
h = cp.Variable(npts)
dual_obj = cp.Maximize(f@xdens + g@ydens - h@bdens)

dual_constraints = []
dual_constraints.append(cp.abs(f) <= 1)
dual_constraints.append(cp.abs(g) <= 1)
dual_constraints.append(cp.abs(h) <= 10)

for k in range(npts):
    for i in range(npts):
        for j in range(npts):
            if max(i,j) == k:
                dual_constraints.append(h[k] >= f[i] + g[j])
                # dual_constraints.append(h[k] <= f[i] + g[j])



dual_prob = cp.Problem(dual_obj, dual_constraints)

# res = dual_prob.solve(solver=cp.ECOS)
res = dual_prob.solve( solver = 'SCIPY', scipy_options = {'method':'highs'})
print(dual_prob.status)
print(dual_prob.value)


In [ ]:
# print values of the strategies
print("Value of marginal strategy", f.value@xdens + g.value@ydens)
print("Value of bestof strategy", h.value@bdens)

In [ ]:
plt.figure()
plt.plot(xgrid, f.value, label = 'f')
plt.plot(xgrid, g.value, '-.', label = 'g')
plt.plot(xgrid, h.value, '-.', label = 'h')
plt.legend(loc='best')
plt.show()


In [11]:
#create 2D versions of the solutions and constraints for plotting

# the grid
sx,sy = np.meshgrid(xgrid,xgrid,indexing = 'ij')

# the f and g functions
sf,sg= np.meshgrid(f.value,g.value,indexing = 'ij')

# This is the sum -- what we can achieve by trading in marginals only
sfg = sf+sg

# This is the payoff that is dominated by sf+sg, a function of the diagonals only
sh = np.zeros((npts, npts))
for i in range(npts):
    for j in range(npts):
        sh[i,j] = h.value[max(i,j)]


In [ ]:

step = 5
fig2 = plt.figure()
ax2 = fig2.add_subplot(projection='3d')
# ax2.plot(sx, sy, sfg, alpha=0.7, cmap = 'YlGnBu', label = 'marginal payoffs')
ax2.scatter(sx[::step,::step], sy[::step,::step], sh[::step,::step], marker = '.', alpha = 1.0, label = 'max payoff')
surf = ax2.plot_surface(sx[::step,::step], sy[::step,::step], sfg[::step,::step], cmap = 'Reds', alpha=0.5, label = 'marginal payoffs')
# ax2.plot_surface(sx, sy, sh, alpha=0.7, label = 'diagonal payoff')

# some magic workaround for the legend() bug, see 
# https://stackoverflow.com/a/54994985/14551426
# does not seem to work
surf._facecolors2d = surf._facecolor3d
surf._edgecolors2d = surf._edgecolor3d

plt.legend(loc = 'upper right')
plt.show()